In [48]:
import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup
import bs4

In [56]:
# Opening the web page to view the HTML source and to understand its structure.
web_page = open("Data/COVID-19 vaccination in the United States - Wikipedia.html","r")

# Reading the page using bs4.
soup = BeautifulSoup(web_page)
web_page.close()

# Finding the table needed for the exercise.
data_tbl = soup.find('table', {'class': 'wikitable sortable static-row-numbers jquery-tablesorter'})

# Getting the headers of table.
headers = [th.getText().strip() for th in data_tbl.findAll('thead')[0].findAll('th')]

# Getting the rows of table and adding to dataframe.
rows = data_tbl.findAll('tbody')[0].findAll('tr')[1:]
d_rows = [[td.get_text().strip() for td in tr.findAll('td')] for tr in rows]
Covid_Vaccinated_by_state_df = pd.DataFrame(d_rows, columns=headers)

# Checking the dataframe.
Covid_Vaccinated_by_state_df.head()

,State/Territory,Vaccinated,% of pop.
0,Alaska,"473,411",64.7%
1,Arizona,"4,858,243",66.7%
2,Arkansas,"1,881,140",62.3%
3,California,"32,416,845",82.0%
4,Colorado,"4,262,561",74.0%


In [58]:
# Step 1:
# Modifying the column name to appropriate name.

Covid_Vaccinated_by_state_df.rename(columns = {'State/Territory':'State','% of pop.':'Pop %'}, inplace = True)

# Checking the columns.
Covid_Vaccinated_by_state_df.columns

Index(['State', 'Vaccinated', 'Pop %'], dtype='object')

In [59]:
# Step 2:
# Adding new state code column. We need it to join with flat file for further analysis.

# Loading states data to dataframe.
states_df = pd.read_csv('Data/States.csv')

# Removing unwanted columns from states dataframe.
states_df = states_df.drop(['Abbrev'], axis = 1)

# Merging two dataframes on state name to get state code.
Covid_Vaccinated_by_state_df = pd.merge(Covid_Vaccinated_by_state_df, states_df, on = 'State')

# Modifying the column name to appropriate name.
Covid_Vaccinated_by_state_df.rename(columns = {'Code':'State Code'}, inplace = True)

# Checking the dataframe.
Covid_Vaccinated_by_state_df.head()

,State,Vaccinated,Pop %,State Code
0,Alaska,"473,411",64.7%,AK
1,Arizona,"4,858,243",66.7%,AZ
2,Arkansas,"1,881,140",62.3%,AR
3,California,"32,416,845",82.0%,CA
4,Colorado,"4,262,561",74.0%,CO


In [61]:
# Step 3:
# Reordering the columns in dataframe.

Covid_Vaccinated_by_state_df = Covid_Vaccinated_by_state_df[['State Code','State','Vaccinated','Pop %']]

# Checking dataframe.
Covid_Vaccinated_by_state_df.head()

,State Code,State,Vaccinated,Pop %
0,AK,Alaska,"473,411",64.7%
1,AZ,Arizona,"4,858,243",66.7%
2,AR,Arkansas,"1,881,140",62.3%
3,CA,California,"32,416,845",82.0%
4,CO,Colorado,"4,262,561",74.0%


In [64]:
# Step 4:
# Identifying outliers/bad data: Checking if there are any null values.

Covid_Vaccinated_by_state_df.isnull().sum()

State Code    0
State         0
Vaccinated    0
Pop %         0
dtype: int64

In [66]:
# Step 5:
# Finding duplicates: Checking if there are any duplicate values in state code.

Covid_Vaccinated_by_state_df['State Code'].duplicated().sum()

0